In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/MyDrive /mydrive
%cd /mydrive/


Mounted at /content/gdrive
/content/gdrive/MyDrive


# Experiments
We'll go through learning feature embeddings using different loss functions on leopard  dataset. We are using 512-dimensional embeddings.

For every experiment Resnet18() is used currently no  hyperparameter search is implemented.

# Prepare dataset
We'll be working on leopard dataset

In [1]:
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.utils.data as data
import torch
transform_img = transforms.Compose([
    transforms.Resize(size= (128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

In [ ]:
%cd /mydrive/siamese-triplet/

cuda = torch.cuda.is_available()
TRAIN_DATA_PATH = '../datasets/leopard/crop/train'
BATCH_SIZE = 8
train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform_img)
TEST_DATA_PATH = '../datasets/leopard/crop/test'
test_dataset = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform_img)

/content/gdrive/MyDrive/siamese-triplet


## Common setup

# Baseline: Classification with softmax
We'll train the model for classification and use outputs of penultimate layer as embeddings

# Online pair/triplet selection - negative mining
There are couple of problems with siamese and triplet networks.
1. The number of possible pairs/triplets grows **quadratically/cubically** with the number of examples. It's infeasible to process them all
2. We generate pairs/triplets randomly. As the training continues, more and more pairs/triplets are easy to deal with (their loss value is very small or even 0), preventing the network from training. We need to provide the network with **hard examples**.
3. Each image that is fed to the network is used only for computation of contrastive/triplet loss for only one pair/triplet. The computation is somewhat wasted; once the embedding is computed, it could be reused for many pairs/triplets.

To deal with that efficiently, we'll feed a network with standard mini-batches as we did for classification. The loss function will be responsible for selection of hard pairs and triplets within mini-batch. In these case, if we feed the network with 16 images per 10 classes, we can process up to $159*160/2 = 12720$ pairs and $10*16*15/2*(9*16) = 172800$ triplets, compared to 80 pairs and 53 triplets in previous implementation.

We can find some strategies on how to select triplets in [2] and [3] *Alexander Hermans, Lucas Beyer, Bastian Leibe, [In Defense of the Triplet Loss for Person Re-Identification](https://arxiv.org/pdf/1703.07737), 2017*

## Online pair selection
## Steps
1. Create **BalancedBatchSampler** - samples $N$ classes and $M$ samples *datasets.py*
2. Create data loaders with the batch sampler
3. Define **embedding** *(mapping)* network $f(x)$ - **EmbeddingNet** from *networks.py*
4. Define a **PairSelector** that takes embeddings and original labels and returns valid pairs within a minibatch
5. Define **OnlineContrastiveLoss** that will use a *PairSelector* and compute *ContrastiveLoss* on such pairs
6. Train the network!

In [ ]:
#from datasets import BalancedBatchSampler
import numpy
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
from datasets import BalancedBatchSampler
train_labels = torch.tensor(train_dataset.targets)
test_labels = torch.tensor(test_dataset.targets)

# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(train_labels, n_classes=64, n_samples=8)
test_batch_sampler = BalancedBatchSampler(test_labels, n_classes=64, n_samples=4)
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)

# Set up the network and training parameters
from networks_mod import EmbeddingNet
from losses import OnlineContrastiveLoss
from utils import AllPositivePairSelector, HardNegativePairSelector # Strategies for selecting pairs within a minibatch

margin = 1.
embedding_net = EmbeddingNet()
model = embedding_net
if cuda:
    model.cuda()
loss_fn = OnlineContrastiveLoss(margin, HardNegativePairSelector())
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 50

In [ ]:
# DO not run Contrastive loss yet
#fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

Train: [0/1337 (0%)]	Loss: 183.808777
Epoch: 1/20. Train set: Average loss: 23.3431
Epoch: 1/20. Validation set: Average loss: 4.0485
Train: [0/1337 (0%)]	Loss: 1.080360
Epoch: 2/20. Train set: Average loss: 0.5589
Epoch: 2/20. Validation set: Average loss: 0.7523
Train: [0/1337 (0%)]	Loss: 0.522320
Epoch: 3/20. Train set: Average loss: 0.3672
Epoch: 3/20. Validation set: Average loss: 0.3205
Train: [0/1337 (0%)]	Loss: 0.421373
Epoch: 4/20. Train set: Average loss: 0.3310
Epoch: 4/20. Validation set: Average loss: 0.2751
Train: [0/1337 (0%)]	Loss: 0.354741
Epoch: 5/20. Train set: Average loss: 0.3092
Epoch: 5/20. Validation set: Average loss: 0.2671
Train: [0/1337 (0%)]	Loss: 0.322270
Epoch: 6/20. Train set: Average loss: 0.2944
Epoch: 6/20. Validation set: Average loss: 0.2592
Train: [0/1337 (0%)]	Loss: 0.311500
Epoch: 7/20. Train set: Average loss: 0.2840
Epoch: 7/20. Validation set: Average loss: 0.2524
Train: [0/1337 (0%)]	Loss: 0.303566
Epoch: 8/20. Train set: Average loss: 0.2790

## Online triplet selection
## Steps
1. Create **BalancedBatchSampler** - samples $N$ classes and $M$ samples *datasets.py*
2. Create data loaders with the batch sampler
3. Define **embedding** *(mapping)* network $f(x)$ - **EmbeddingNet** from *networks.py*
4. Define a **TripletSelector** that takes embeddings and original labels and returns valid triplets within a minibatch
5. Define **OnlineTripletLoss** that will use a *TripletSelector* and compute *TripletLoss* on such pairs
6. Train the network!

In [ ]:
import numpy
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
from datasets import BalancedBatchSampler
train_labels = torch.tensor(train_dataset.targets)
test_labels = torch.tensor(test_dataset.targets)

# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(train_labels, n_classes=64, n_samples=8)
test_batch_sampler = BalancedBatchSampler(test_labels, n_classes=64, n_samples=4)
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)

# Set up the network and training parameters
from networks_mod import EmbeddingNet
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = EmbeddingNet()
model = embedding_net
if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 50

In [ ]:
fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

Train: [0/1337 (0%)]	Loss: 0.917869	Average nonzero triplets: 1513.0
Epoch: 1/20. Train set: Average loss: 0.8143	Average nonzero triplets: 1513.0
Epoch: 1/20. Validation set: Average loss: 0.8925	Average nonzero triplets: 183.0
Train: [0/1337 (0%)]	Loss: 0.820511	Average nonzero triplets: 1513.0
Epoch: 2/20. Train set: Average loss: 0.7932	Average nonzero triplets: 1513.0
Epoch: 2/20. Validation set: Average loss: 0.9022	Average nonzero triplets: 183.0
Train: [0/1337 (0%)]	Loss: 0.726920	Average nonzero triplets: 1513.0
Epoch: 3/20. Train set: Average loss: 0.7169	Average nonzero triplets: 1513.0
Epoch: 3/20. Validation set: Average loss: 0.8005	Average nonzero triplets: 183.0
Train: [0/1337 (0%)]	Loss: 0.708016	Average nonzero triplets: 1513.0
Epoch: 4/20. Train set: Average loss: 0.7177	Average nonzero triplets: 1513.0
Epoch: 4/20. Validation set: Average loss: 0.7522	Average nonzero triplets: 183.0
Train: [0/1337 (0%)]	Loss: 0.719246	Average nonzero triplets: 1513.0
Epoch: 5/20. Tr

In [ ]:
model_file_name = 'leopard_model_tr.pt'
path = f"/content/gdrive/MyDrive/siamese-triplet/{model_file_name}" 
torch.save(model.state_dict(), path)

In [ ]:
def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = []
        labels = []
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings.extend(model.get_embedding(images).data.cpu().numpy())
            labels.extend(target.numpy().tolist())
    return embeddings, labels

In [ ]:
train_eval_loader = data.DataLoader(train_dataset, batch_size=4, shuffle=False,  num_workers=2, drop_last=True, pin_memory=cuda)
train_emb, train_ref_label = extract_embeddings(train_eval_loader, model)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:

test_eval_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=False,  num_workers=2, drop_last=True, pin_memory=cuda)
test_emb, test_ref_label = extract_embeddings(test_eval_loader, model)

/content/gdrive/MyDrive/siamese-triplet
